In [2]:
import tensorflow
import sklearn
from tensorflow.keras.datasets import reuters
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
print(tensorflow.__version__)
# print(matplotlib.__version__)
print(sns.__version__)
print(np.__version__)
print(pd.__version__)
print(sklearn.__version__)

2.6.0
0.11.2
1.21.4
1.3.3
1.0


### 데이터 처리


In [3]:
#로이터 뉴스 데이터를 훈련 데이터와 테스트 데이터로 나누어 변수에 각각 저장합니다.
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=10000, test_split=0.2)

#데이터가 어떤 구성을 가지고 있는지 출력
print('훈련 샘플의 수: {}'.format(len(x_train)))
print('테스트 샘플의 수: {}'.format(len(x_test)))

print('훈련 샘플 레이블의 수: {}'.format(len(y_train)))
print('테스트 샘플 레이블의 수: {}'.format(len(y_test)))

#뉴스 데이터를 다루기로 했는데, 실제 출력해보면 텍스트가 아니라 숫자 시퀀스가 출력
#각 단어가 빈도수가 높은 순서대로 낮은 정수가 맵핑
print(x_train[0])
print(x_test[0])

#클래스 확인
num_classes = max(y_train) + 1
print('클래스의 수 : {}'.format(num_classes))

#로이터 뉴스 데이터는 '단어'를 key값으로, 고유한 '정수'를 value로 가지는 dictionary를 제공합니다. 이를 word_index로 저장
word_index = reuters.get_word_index(path="reuters_word_index.json")
word_index['it']


#word_index보다는 정수로부터 단어를 얻을 수 있는index_word가 필요
#word_index의 결과로 나오는 숫자에 ***3을 더해주어야*** 단어가 실제로 맵핑된 인덱스 값이 나옵
index_to_word = { index+3 : word for word, index in word_index.items() }
#13+3
print('index_to_word: ', index_to_word[16])

#0번, 1번, 2번 인덱스에는 각각 , , 라는 특별한 토큰이 맵핑되어 있다고 했죠? 그래서 만들어진 index_to_word에 이 토큰들도 추가해주어야 진짜 index_to_word가 완성
# index_to_word에 숫자 0은 <pad>, 숫자 1은 <sos>, 숫자 2는 <unk>를 넣어줍니다.
for index, token in enumerate(("<pad>", "<sos>", "<unk>")):
  index_to_word[index]=token

print(' '.join([index_to_word[index] for index in x_train[0]]))

#전체 훈련용 뉴스 데이터와 전체 테스트용 뉴스 데이터를 텍스트 데이터로 변환
decoded = []
for i in range(len(x_train)):
    t = ' '.join([index_to_word[index] for index in x_train[i]])
    decoded.append(t)

x_train = decoded
print('x_train 길이:', len( x_train))

decoded = []
for i in range(len(x_test)):
    t = ' '.join([index_to_word[index] for index in x_test[i]])
    decoded.append(t)

x_test = decoded
print('x_test 길이:', len(x_test))


훈련 샘플의 수: 8982
테스트 샘플의 수: 2246
훈련 샘플 레이블의 수: 8982
테스트 샘플 레이블의 수: 2246
[1, 2, 2, 8, 43, 10, 447, 5, 25, 207, 270, 5, 3095, 111, 16, 369, 186, 90, 67, 7, 89, 5, 19, 102, 6, 19, 124, 15, 90, 67, 84, 22, 482, 26, 7, 48, 4, 49, 8, 864, 39, 209, 154, 6, 151, 6, 83, 11, 15, 22, 155, 11, 15, 7, 48, 9, 4579, 1005, 504, 6, 258, 6, 272, 11, 15, 22, 134, 44, 11, 15, 16, 8, 197, 1245, 90, 67, 52, 29, 209, 30, 32, 132, 6, 109, 15, 17, 12]
[1, 4, 1378, 2025, 9, 697, 4622, 111, 8, 25, 109, 29, 3650, 11, 150, 244, 364, 33, 30, 30, 1398, 333, 6, 2, 159, 9, 1084, 363, 13, 2, 71, 9, 2, 71, 117, 4, 225, 78, 206, 10, 9, 1214, 8, 4, 270, 5, 2, 7, 748, 48, 9, 2, 7, 207, 1451, 966, 1864, 793, 97, 133, 336, 7, 4, 493, 98, 273, 104, 284, 25, 39, 338, 22, 905, 220, 3465, 644, 59, 20, 6, 119, 61, 11, 15, 58, 579, 26, 10, 67, 7, 4, 738, 98, 43, 88, 333, 722, 12, 20, 6, 19, 746, 35, 15, 10, 9, 1214, 855, 129, 783, 21, 4, 2280, 244, 364, 51, 16, 299, 452, 16, 515, 4, 99, 29, 5, 4, 364, 281, 48, 10, 9, 1214, 23, 644, 

### 벡터화 하기



In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

#학습 데이터에 대한DTM을 생성하고, DTM의 크기를 확인
dtmvector = CountVectorizer()
x_train_dtm = dtmvector.fit_transform(x_train)
print('x_train_dtm.shape:',x_train_dtm.shape)

#이상한 점은 앞에서 데이터를 로드할 때, num_words=10,000이라는 값을 사용했음에도 DTM 열의 개수는 이보다 현저하게 적은 9,670개밖에 되지 않습니다. 
#그 이유는 DTM이 자체적인 규칙에 따라서 불필요하다고 판단하는 토큰들을 제거하기 때문

#TF-IDF Matrix는 사이킷런의 TfidfTransformer()를 통해서 생성할 수 있습니다. 
#TF-IDF Matrix는 추가적인 전처리를 하지 않는 이상, DTM과 동일한 크기를 가짐
tfidf_transformer = TfidfTransformer()
tfidfv = tfidf_transformer.fit_transform(x_train_dtm)
print('tfidfv.shape:',tfidfv.shape)

#테스트 데이터를 DTM으로 변환
x_test_dtm = dtmvector.transform(x_test) #테스트 데이터를 DTM으로 변환
tfidfv_test = tfidf_transformer.transform(x_test_dtm) #DTM을 TF-IDF 행렬로 변환
print('tfidfv_test.shape:',tfidfv_test.shape)

x_train_dtm.shape: (8982, 9670)
tfidfv.shape: (8982, 9670)
tfidfv_test.shape: (2246, 9670)


### 다양한 모델들

**Complement Naive Bayes Classifier(CNB)**

---

In [5]:
from sklearn.naive_bayes import MultinomialNB #다항분포 나이브 베이즈 모델
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import ComplementNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score #정확도 계산

In [6]:
cb = ComplementNB()
cb.fit(tfidfv, y_train)


ComplementNB()

In [7]:
predicted = cb.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("ComplementNB의 정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

ComplementNB의 정확도: 0.7707034728406055


**로지스틱 회귀(Logistic Regression)**

---



In [8]:
lr = LogisticRegression(C=10000, penalty='l2', max_iter=3000)
lr.fit(tfidfv, y_train)

LogisticRegression(C=10000, max_iter=3000)

In [9]:
predicted = lr.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("LogisticRegression의 정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

LogisticRegression의 정확도: 0.8107747105966162


**랜덤 포레스트(Random Forest)**

---

In [10]:
forest = RandomForestClassifier(n_estimators=5, random_state=0)
forest.fit(tfidfv, y_train)

RandomForestClassifier(n_estimators=5, random_state=0)

In [11]:
predicted = forest.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("RandomForestClassifier의 정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

RandomForestClassifier의 정확도: 0.674087266251113


**그래디언트 부스팅 트리(GradientBoostingClassifier)**

---

In [12]:
grbt = GradientBoostingClassifier(random_state=0) # verbose=3
grbt.fit(tfidfv, y_train)

GradientBoostingClassifier(random_state=0)

In [13]:
predicted = grbt.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("GradientBoostingClassifier의 정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

GradientBoostingClassifier의 정확도: 0.7662511130899377


**보팅(Voting)**

---


In [14]:
log_clf = LogisticRegression(penalty='l2', random_state=0)

# Complement Naive Bayes Classifier
cnb_clf = ComplementNB()

# Gradient Boosting Classifier
gb_clf = GradientBoostingClassifier(random_state=0)

voting_classifier = VotingClassifier(
    estimators=[
        ('lr', log_clf),
        ('cnb', cnb_clf),
        ('gb', gb_clf)
    ],
    voting='hard'
)
voting_classifier.fit(tfidfv, y_train)

/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


VotingClassifier(estimators=[('lr', LogisticRegression(random_state=0)),
                             ('cnb', ComplementNB()),
                             ('gb',
                              GradientBoostingClassifier(random_state=0))])

In [15]:
predicted = voting_classifier.predict(tfidfv_test) #테스트 데이터에 대한 예측
print("voting_classifier의 정확도:", accuracy_score(y_test, predicted)) #예측값과 실제값 비교

voting_classifier의 정확도: 0.7991985752448798


## 테스트 결과

**테스트 케이스별 정확도**

---
1. num_word = 10000
   * ComplementNB의 정확도: 0.7707034728406055
   * LogisticRegression의 정확도: 0.8085485307212823
   * RandomForestClassifier의 정확도: 0.674087266251113
   * GradientBoostingClassifier의 정확도: 0.7671415850400712
   * voting_classifier의 정확도: 0.8000890471950134(hard), 0.7898486197684773(soft)
   * 
   --> RandomForest(n_estimators=10(5-->10))

      RandomForestClassifier의 정확도: 0.7123775601068566

      RandomForest(n_estimators=10(5-->20))

      RandomForestClassifier의 정확도: 0.7368655387355298
 
2. num_word = 5000
   * ComplementNB의 정확도: 0.7707034728406055
   * LogisticRegression의 정확도: 0.8063223508459484
   * RandomForestClassifier의 정확도: 0.6999109528049866
   * GradientBoostingClassifier의 정확도: 0.7658058771148709
   * voting_classifier의 정확도: 0.7991985752448798
   * 
   --> RandomForest(n_estimators=10(5-->10))
   
          RandomForestClassifier의 정확도: 0.7270703472840605
   
       RandomForest(n_estimators=10(5-->20))
   
        RandomForestClassifier의 정확도: 0.7497773820124666

## 회고

lecture 기준 파라미터로 각 모델에 따른 테스트에서는 RadomForest가 가장 낮은 수치를 기록 하였습니다. 

RandForest 모델의 결과 중 특이 사항은

  * n_estimators(5-->10-->20) 수치를 증가 시켜면 정확도는 비례하여 증가한다.
  
  * num_words이 수치를 감소(10000-->5000) 시키면 정확도는 반비례하여 증가한다.

Voting을 'soft'로 변환하면 더 hard보다 더 낮은 정확도를 보였습니다.

다중 분류에서 트리 모델인 RandomForest 가 정확도가 모델 중 가장 낮은 수치를 보였는데,

n_estimators 수치를 증가 시키면 정확도는 비례 하여 증가 하였습니다.

몇 개 정도 설정 하면 RandomForest  정확도에 유사 하게 접근 할 수 있는지 테스트 해 보는 것도 좋을 것 같습니다.

Voting이 정확도가 가장 높지 않을까 예측 했는데, LogisticRegression을 뛰어 넘지 않아,

문제에 맞는 모델을 사용하는 것이 중요 하다고 느꼈습니다.